In [57]:

from tkinter import * 
import requests
import json

root = Tk()

import sys
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
import time
import requests
# Set up authorization credentials
client_id = '4fdf0c9814f04bf2a858554336140450'
client_secret = '5512f23f0a9048f28140e217c5db1b29'
genius_access_token = "Bjc_l29KgRjxlf7pNJ-vM7zEDKG5xbf12fJ4K1TDZdWKW2jjh87Jqy7wz38ocUCu"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
genius = lyricsgenius.Genius(genius_access_token)
# Search for a song on Spotify based on lyrics

global spotify_urisandnames
spotify_urisandnames = {}

def get_spotify_uri_from_lyrics(lyrics):
    # search for the song title and artist using the lyricsgenius package
    song = genius.search_song(lyrics)
    if not song:
        return 'Could not find song with the given lyrics'
    # search for the song on Spotify using the spotipy package
    result = sp.search(q='{} {}'.format(song.title, song.artist), type='track')
# extract the Spotify URIs of the songs from the search results
    spotify_uris = []
    for item in result['tracks']['items']:
        spotify_uri = item['uri']
        spotify_name = item["name"]
        spotify_urisandnames[spotify_uri] = spotify_name
        spotify_uris.append(spotify_uri)
    
    # return the list of Spotify URIs
    if len(spotify_uris) > 0:
        print('Spotify URIs: {}'.format(', '.join(spotify_uris)))
        return spotify_uris
    else:
        print('Could not find song on Spotify')
        
def get_lyrics_from_uri(uri):
    # extract the track ID from the Spotify URI
    track_id = uri.split(':')[-1]

    # use the Spotipy package to get the track's information
    track = sp.track(track_id)

    # use the LyricsGenius package to fetch the lyrics
    lyrics = genius.search_song(track['name'], track['artists'][0]['name'])

    if lyrics is not None:
        return lyrics.lyrics
    else:
        return 'No lyrics found for this song'
    

#set variable equal to 0 so the top search result is printed first

#GUI BUILDING

def backone():
    global numsong, uri_list
    if numsong > 0 and numsong <= len(uri_list):
        numsong -= 1
    else:
        textbox.delete(0.0, END)
        textbox.insert(END, none)
    inputlyr = lyricvar.get() 
    uri_list = get_spotify_uri_from_lyrics(inputlyr)
    textbox.delete(0.0, END)
    textbox.insert(END, get_lyrics_from_uri(uri_list[numsong]))
    titlevar.set(spotify_urisandnames[uri_list[numsong]])
    lastsong.grid(row = 9, column = 2)
    nextsong.grid(row = 9, column = 6)
                   
def submitlyrics():
    global numsong, uri_list
    numsong = 0
    inputlyr = lyricvar.get() 
    uri_list = get_spotify_uri_from_lyrics(inputlyr)
    textbox.delete(0.0, END)
    textbox.insert(END, get_lyrics_from_uri(uri_list[numsong]))
    titlevar.set(spotify_urisandnames[uri_list[numsong]])    
    lastsong.grid(row = 9, column = 2)
    nextsong.grid(row = 9, column = 6)
    
    
def nextone():
    global numsong, uri_list
    if numsong < len(uri_list):
        numsong += 1
    else:
        textbox.delete(0.0, END)
        textbox.insert(END, none)
    inputlyr = lyricvar.get() 
    uri_list = get_spotify_uri_from_lyrics(inputlyr)
    textbox.delete(0.0, END)
    textbox.insert(END, get_lyrics_from_uri(uri_list[numsong]))
    titlevar.set(spotify_urisandnames[uri_list[numsong]])
    lastsong.grid(row = 9, column = 2)
    nextsong.grid(row = 9, column = 6)
             


root.title("Lyric Match")
root.geometry("900x1000")
root.configure(background='DarkSeaGreen1')


lyricvar = StringVar()
titlevar = StringVar()

none = "No More Songs Found!"
instruct_text1 = "Welcome to LyricMatch!" 
instruct_text2 = "Type a lyric thats on your mind in the box below, click the 'Submit Lyrics' button, and we'll try to give you a range of matches for them!" 
instruct_text3 = "Two buttons will appear when you submit your lyrics, as well as the title of the song. The 'next song' and 'previous song' buttons allow for you to cycle through them!"
instruct_text4 = "Keep in mind, it may take a second for lyrics to appear."



textbox = Text(root, font = "Ariel", height = 30, width = 40)
titlelabel = Label(root, textvariable = titlevar, font = ("Ariel", 14, "bold"))
lyrics = Entry(root, textvariable = lyricvar)
submitbutton = Button(root, text = "Submit Lyrics", command = submitlyrics)
instructions1 = Label(root, text = instruct_text1)
instructions2 = Label(root, text = instruct_text2)
instructions3 = Label(root, text = instruct_text3)
instructions4 = Label(root, text = instruct_text4)
nextsong = Button(root, text = "Next Song", command = nextone)
lastsong = Button(root, text = "Previous Song", command = backone)

instructions1.grid(row = 1, column = 4)
instructions2.grid(row = 2, column = 4)
instructions3.grid(row = 3, column = 4)
instructions4.grid(row = 4, column = 4)
titlelabel.grid(row = 5, column = 4)
lyrics.grid(row = 6, column = 4)
textbox.grid(row = 7, column = 4)
submitbutton.grid(row = 8, column = 4)

root.mainloop()


